In [78]:
# imports
import pandas as pd
import seaborn as sns
import numpy as np

## Data preprocessing

### Import

In [47]:
movie_df = pd.read_csv("../oscar_movies.csv")
movie_df

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,...,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
0,2001,Kate & Leopold,tt0035423,PG-13,118,Comedy|Fantasy|Romance,6.4,44.0,An English Duke from 1876 is inadvertedly drag...,66660,...,0,NaN,0,NaN,0,NaN,2001.0,12.0,25.0,2.0
1,2000,Chicken Run,tt0120630,G,84,Animation|Adventure|Comedy,7.0,88.0,When a cockerel apparently flies into a chicke...,144475,...,1,Best Animated Film,1,Best Animation,1,Best Animation,2000.0,6.0,23.0,5.0
2,2005,Fantastic Four,tt0120667,PG-13,106,Action|Adventure|Family,5.7,40.0,A group of astronauts gain superpowers after a...,273203,...,0,NaN,0,NaN,0,NaN,2005.0,7.0,8.0,5.0
3,2002,Frida,tt0120679,R,123,Biography|Drama|Romance,7.4,61.0,"A biography of artist Frida Kahlo, who channel...",63852,...,0,NaN,0,NaN,0,NaN,2002.0,11.0,22.0,5.0
4,2001,The Lord of the Rings: The Fellowship of the Ring,tt0120737,PG-13,178,Adventure|Drama|Fantasy,8.8,92.0,A meek Hobbit from the Shire and eight compani...,1286275,...,0,NaN,1,Best Music,2,Best Music|Best Production Design,2001.0,12.0,19.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,2017,Call Me by Your Name,tt5726616,R,132,Drama|Romance,8.3,93.0,"In Northern Italy in 1983, seventeen year-old ...",38170,...,1,Best Actor,3,Best Actor|Best Director|Best Picture,3,Best Actor|Best Director|Best Picture,NaN,NaN,NaN,NaN
1179,2017,Phantom Thread,tt5776858,R,130,Drama|Romance,8.4,90.0,"Set in 1950's London, Reynolds Woodcock is a r...",7380,...,1,Best Screenplay,1,Best Music,1,Best Music,NaN,NaN,NaN,NaN
1180,2017,Victoria & Abdul,tt5816682,PG-13,111,Biography|Drama|History,6.8,58.0,Queen Victoria strikes up an unlikely friendsh...,12888,...,0,NaN,0,NaN,0,NaN,2017.0,10.0,6.0,5.0
1181,2017,"Roman J. Israel, Esq.",tt6000478,PG-13,122,Crime|Drama,6.3,58.0,"Roman J. Israel, Esq., a driven, idealistic de...",3205,...,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


### Process

Basic check to make sure IDs are unique, since movies can very much so have the same name (as shown).

In [48]:
# make sure all IDs are unique

assert len(movie_df[movie_df.duplicated()]) == 0 

movie_df[movie_df.duplicated("movie", keep=False)] # this is OK, they are real movies with the same titles. 

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,...,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
484,2006,The Illusionist,tt0443543,PG-13,110,Drama|Mystery|Romance,7.6,68.0,"In turn-of-the-century Vienna, a magician uses...",304063,...,0,NaN,0,NaN,0,NaN,2006.0,9.0,1.0,5.0
588,2010,The Illusionist,tt0775489,PG,80,Animation|Drama,7.5,82.0,A French illusionist finds himself out of work...,29236,...,1,Best Animated Film,0,NaN,1,Best Animation,2011.0,2.0,11.0,5.0


Oscar category wins have their own columns with "Yes" or "No" flags.
These flags should be booleans. 

In [49]:
# convert Oscar flags to boolean instead of "Yes", "No" (all starting with "Oscar_" except "Oscar_nominated","Oscar_nominated_categories")
for column in movie_df.columns:
    # get relevant Oscar flag columns
    if column.startswith("Oscar_") and column not in ["Oscar_nominated", "Oscar_nominated_categories"]:
        movie_df[column] = movie_df[column].replace({"Yes": True, "No": False})

movie_df["Oscar_Best_Picture_won"].dtype

dtype('bool')

Metascore (metacritic score) is an integer. But the `"metascore"` column is a float! Fix that.

In [50]:
# convert metascore (which is an int) to int.
movie_df["metascore"] = pd.to_numeric(movie_df["metascore"], errors='coerce').astype('Int64')

movie_df[movie_df["metascore"].isna()][["movie"]] # 13 movies are missing metascores. Unpopular movies. 

,movie
88,Ken Park
272,Twin Sisters
348,The Room
380,As It Is in Heaven
450,Yesterday
474,The Queen
577,Beaufort
652,Unthinkable
669,"Angus, Thongs and Perfect Snogging"
700,Death Proof


Popularity is based on the date the movie was scraped. 1 is the most popular movie on IMDB of that time. 
There are several movies in this dataset without a popularity statistic. Because the score is based on when it was scraped, we cannot manually fill these in.

'Moana' being there suggests to me that it's probably not exclusively very unpopular movies.

In [75]:
print(sum(movie_df["popularity"].isna()))
print(movie_df[movie_df["popularity"].isna()][["movie"]])

119
                           movie
25             Nowhere in Africa
54                       Pollock
56                   Wonder Boys
64         Shadow of the Vampire
67                         Vatel
...                          ...
1116             A Monster Calls
1121                        Sing
1124                       Moana
1130       The Girl on the Train
1171  Jim: The James Foley Story

[119 rows x 1 columns]


Luckily, movies released before 2017's US & Canada box grosses have not changed much from then to now, so the stats can be manually filled in.

In [52]:
# fill in missing grosses manually
# grosses are found on IMDB's Gross US & Canada rounded to the nearest $10k

gross_fill_dict = { # manually built from IMDB website (imdb.com)
    'Lagaan: Once Upon a Time in India': 909043,
    # 'Ken Park': 0, (missing US & Canada data)
    'Zus & zo': 49468,
    'The Twilight Samurai': 559765,
    'The Room': 549602,
    # 'Yesterday': 0, (missing US & Canada data)
    # 'Unthinkable': 0, (missing US & Canada data)
    # 'Angus, Thongs and Perfect Snogging': 0, (missing US & Canada data)
    # 'Death Proof': 0, (missing US & Canada data)
    # "Hachi: A Dog's Tale": 0, (missing US & Canada data)
    'Revanche': 258388,
    'Outside the Law': 96933,
    'Secret in Their Eyes': 20180155,
    'The Great Wall': 45540830,
    # 'Mudbound': 0, (missing US & Canada data)
    'Leviathan': 1092800,
    'Omar': 356000,
    'Trumbo': 7857741,
    'The Autopsy of Jane Doe': 10474,
    'Elle': 2341534,
    'Land of Mine': 435266,
    # 'The Girl with All the Gifts': 0, (missing US & Canada data)
    'Split': 138291365,
    # 'Jim: The James Foley Story': 0 (missing US & Canada data)
}
# round to $10k
gross_fill_dict = {key: round(value, -4) for key, value in gross_fill_dict.items()}


movie_df.loc[movie_df["gross"].isna(), "gross"] = movie_df["movie"].map(gross_fill_dict)

assert sum(movie_df["gross"].isna()) == 9

movie_df[movie_df["gross"].isna()][["year", "movie", "gross"]]

,year,movie,gross
88,2002,Ken Park,NaN
450,2004,Yesterday,NaN
652,2010,Unthinkable,NaN
669,2008,"Angus, Thongs and Perfect Snogging",NaN
700,2007,Death Proof,NaN
701,2009,Hachi: A Dog's Tale,NaN
1045,2017,Mudbound,NaN
1158,2016,The Girl with All the Gifts,NaN
1171,2016,Jim: The James Foley Story,NaN


Sanity check on votes.

In [77]:
print(sum(movie_df["votes"].isna()))

movie_with_most_votes = movie_df.loc[movie_df["votes"].idxmax(), "movie"]
movie_with_most_votes # The Dark Knight seems right.

0


'The Dark Knight'

#### Release dates

Make sure that missing `release_date` means missing `release_date.year`, `release_date.month`, `release_date.day-of-month`, and `release_date.day-of-week`. (It does!)

In [54]:
# check if missing release dates corresponds to missing release date specifics

no_release_date = movie_df["release_date"].isna()

no_year = (movie_df["release_date.year"].isna())
no_month = (movie_df["release_date.month"].isna())
no_day_month = (movie_df["release_date.day-of-month"].isna())
no_day_week = (movie_df["release_date.day-of-week"].isna())

# assert all refer to the same rows
assert ((no_release_date == no_year).all() 
        and (no_year == no_month).all() 
        and (no_month == no_day_month).all() 
        and (no_day_month == no_day_week).all())

# get movies without entered release dates
movie_df[movie_df["release_date"].isna()]["movie"]

1045                                     Mudbound
1060                                       Wonder
1165                                    Lady Bird
1169    Three Billboards Outside Ebbing, Missouri
1173                                     Marshall
1177                          The Florida Project
1178                         Call Me by Your Name
1179                               Phantom Thread
1181                        Roman J. Israel, Esq.
Name: movie, dtype: object

Go to datetime tipe for use later.

In [55]:
# convert "release_date" column to datetime types
movie_df["release_date"] = pd.to_datetime(movie_df["release_date"], dayfirst=False)

movie_df["release_date"].dtype
# dtype('<M8[ns]') is a datetype type

dtype('<M8[ns]')

Missing release dates are fixed manually using IMDB release dates to match the rest of the database. 

In [56]:
# fix missing release dates

movie_release_dict = {
    "Mudbound": pd.to_datetime("2017-11-17", yearfirst=True, dayfirst=False),
    "Wonder": pd.to_datetime("2017-11-17", yearfirst=True, dayfirst=False),
    "Lady Bird": pd.to_datetime("2017-12-1", yearfirst=True, dayfirst=False),
    "Three Billboards Outside Ebbing, Missouri": pd.to_datetime("2017-12-1", yearfirst=True, dayfirst=False),
    "Marshall": pd.to_datetime("2017-10-13", yearfirst=True, dayfirst=False),
    "The Florida Project": pd.to_datetime("2017-11-10", yearfirst=True, dayfirst=False),
    "Call Me by Your Name": pd.to_datetime("2018-1-19", yearfirst=True, dayfirst=False),
    "Phantom Thread": pd.to_datetime("2018-1-19", yearfirst=True, dayfirst=False),
    "Roman J. Israel, Esq.": pd.to_datetime("2017-11-22", yearfirst=True, dayfirst=False),
}

# fill missing release dates
no_release_date = movie_df["release_date"].isna()
movie_df.loc[no_release_date, "release_date"] = movie_df["movie"].map(movie_release_dict)

# Extract year, month, and day only for the rows in no_release_date
movie_df.loc[no_release_date, "release_date.year"] = movie_df["release_date"].dt.year
movie_df.loc[no_release_date, "release_date.month"] = movie_df["release_date"].dt.month
movie_df.loc[no_release_date, "release_date.day-of-month"] = movie_df["release_date"].dt.day

# NOTE: datetime package uses 0 as Monday, 6 as Sunday. CSV uses 1 as Monday and 7 as Sunday.  
# When adding new weekdays, *for now* we will use the CSV's form so when alignment with datetime happens they're all misaligned in the same way.
movie_df.loc[movie_df["release_date.day-of-week"].isna(), "release_date.day-of-week"] = movie_df["release_date"].dt.weekday + 1

In [57]:
movie_df[ # convert "release_date.____" columns to ints instead of floats
    ["release_date.year", "release_date.month", "release_date.day-of-month", "release_date.day-of-week"]
    ] = movie_df[
        ["release_date.year", "release_date.month", "release_date.day-of-month", "release_date.day-of-week"]
        ].astype(int)
    
# align "release_date.day-of-week"
"""
0: mon
1: tues
2: wed
3: thurs
4: fri (most common)
5: sat
6: sun
"""

movie_df["release_date.day-of-week"] -= 1

In [58]:
# inspect manually
movie_df[["release_date", "release_date.year", "release_date.month", "release_date.day-of-month", "release_date.day-of-week"]]

,release_date,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
0,2001-12-25,2001,12,25,1
1,2000-06-23,2000,6,23,4
2,2005-07-08,2005,7,8,4
3,2002-11-22,2002,11,22,4
4,2001-12-19,2001,12,19,2
...,...,...,...,...,...
1178,2018-01-19,2018,1,19,4
1179,2018-01-19,2018,1,19,4
1180,2017-10-06,2017,10,6,4
1181,2017-11-22,2017,11,22,2


#### Make lists lists

In [59]:
# convert "categories" nominated/won from "|"-delimited strings to lists of strings
for column in movie_df.columns:
    if "categories" in column:
        # convert to list if string, if already list then keep as list, otherwise empty list (no nominations)
        movie_df[column] = movie_df[column].apply(lambda entry: entry.split("|") if isinstance(entry, str) else (entry if isinstance(entry, list) else []))

# sanity check: make sure each entry length matches (for oscar at least)
oscar_mismatch_rows = movie_df[movie_df['Oscar_nominated_categories'].apply(len) != movie_df['Oscar_nominated']]
oscar_mismatch_rows[["Oscar_nominated_categories", "Oscar_nominated"]]

,Oscar_nominated_categories,Oscar_nominated


In [60]:
# convert "genre" from "|"-delimited strings into lists of strings
assert movie_df["genre"].isna().sum() == 0 # luckily, every movie has a genre already

# convert to list if string, if already list then keep as list, otherwise empty list (should never happen)
movie_df["genre"] = movie_df["genre"].apply(lambda entry: entry.split("|") if isinstance(entry, str) else (entry if isinstance(entry, list) else []))

assert all (movie_df["genre"].apply(lambda x: len(x) > 0 and isinstance(x, list))) # confirm all entries still have a genre and that all are lists

movie_df["genre"]

0           [Comedy, Fantasy, Romance]
1       [Animation, Adventure, Comedy]
2          [Action, Adventure, Family]
3          [Biography, Drama, Romance]
4          [Adventure, Drama, Fantasy]
                     ...              
1178                  [Drama, Romance]
1179                  [Drama, Romance]
1180       [Biography, Drama, History]
1181                    [Crime, Drama]
1182       [Biography, Drama, History]
Name: genre, Length: 1183, dtype: object

In [61]:
movies_with_zero_length_genre_element = movie_df[movie_df["genre"].apply(lambda x: any(len(genre) == 0 for genre in x))]
movies_with_zero_length_genre_element

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,...,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week


In [73]:
# Numerous '_categories' entries are missing. Too numerous to add manually.
# However, the '_won' and '_nominated' *non '_category'* columns appear accurate following manual investigation.

# Bafta
movies_with_zero_length_bafta_element = movie_df[movie_df["BAFTA_nominated_categories"].apply(lambda x: any(len(elem) == 0 for elem in x))]
movies_with_zero_length_bafta_element[["movie", "BAFTA_won", "BAFTA_won_categories", "BAFTA_nominated", "BAFTA_nominated_categories"]]
print(len(movies_with_zero_length_bafta_element))

# LAFC
movies_with_zero_length_lafc_element = movie_df[movie_df["Los_Angeles_Film_Critics_Association_nominated_categories"].apply(lambda x: any(len(elem) == 0 for elem in x))]
movies_with_zero_length_lafc_element[["movie", "Los_Angeles_Film_Critics_Association_won", "Los_Angeles_Film_Critics_Association_won_categories", "Los_Angeles_Film_Critics_Association_nominated", "Los_Angeles_Film_Critics_Association_nominated_categories"]]
print(len(movies_with_zero_length_lafc_element))

# Austin
movies_with_zero_length_austin_element = movie_df[movie_df["Austin_Film_Critics_Association_nominated_categories"].apply(lambda x: any(len(elem) == 0 for elem in x))]
movies_with_zero_length_austin_element[["movie", "Austin_Film_Critics_Association_won", "Austin_Film_Critics_Association_won_categories", "Austin_Film_Critics_Association_nominated", "Austin_Film_Critics_Association_nominated_categories"]]
print(len(movies_with_zero_length_austin_element))

# Hollywood
movies_with_zero_length_hollywood_element = movie_df[movie_df["Hollywood_Film_nominated_categories"].apply(lambda x: any(len(elem) == 0 for elem in x))]
movies_with_zero_length_hollywood_element[["movie", "Hollywood_Film_won", "Hollywood_Film_won_categories", "Hollywood_Film_nominated", "Hollywood_Film_nominated_categories"]]
print(len(movies_with_zero_length_hollywood_element))

103
6
12
9


### Summary Stats

In [ ]:
movie_df.describe()

## Investigations:

- [First — TBD](first_investigation.ipynb)
- [Second — TBD](second_investigation.ipynb)
- [Third — TBD](third_investigation.ipynb)